### Project description:
#### Goals of analysis
* 1. Geospatial analysis of nepal earthquake data 2015
* 2. Local Boundaries brings the detailed geodata of administrative units or maps of all administrative boundary defined by Nepal Government, in open and reusable format, free of cost. The local boundaries are available in two formats (TopoJSON and GeoJSON) and can be easily reused to map local authority data to OpenStreetMap, Google Map, Leaflet or MapBox interactively.The data of Local Boundaries are categorized in three different ways following the new federal structure, with 7 Provinces, 77 Districts, and 753 Local Levels. You can access the TopoJSON and GeoJSON files of all local levels, separately or in bulk according to your needs. http://localboundries.oknp.org/?fbclid=IwAR3Es1qk6DqarBO3vQERdfP7ntkS7Tp57kADlAueuuyb-EmLUJCMMMyLjIU
* 3. The NHRP API requires one to provide a location_code. Tables below should help you with that, by providing location codes at a district and municipality level. Ward level codes are generated by appending the 2-digit ward number (padded with zeroes) to a municipality code (eg, Ward 2, Champadevi Rural Municipality is simply 1201 + 02 = 120102).
http://eq2015.npc.gov.np/docs/?fbclid=IwAR2Bg_M2GRYCj_jkdbAOB-9Td-k40wsg0dchdS9Lm8TDv8_cHeK_vQM7oYs#/location_codes
* 4. Understanding of shapefile https://gisgeography.com/arcgis-shapefile-files-types-extensions/
* 5. scipy https://github.com/kjordahl/SciPy2013

#### Initializing packages

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pygeoj
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import defaultdict
from sklearn.model_selection import train_test_split
from utils import *
warnings.filterwarnings("ignore")

#### Loading files
* combined clean data of building structure ownership
* ward mapping district data 
* lat long data of all nepal districts - combined with ward-mapping files

In [4]:
BuildingPath      = "input_data/building_damage_assessment_building_ownership_and_use_building_structure/"
data              = pd.read_csv('clean_data/building_structure_ownership.csv')
ward_mapping_data = pd.read_csv(BuildingPath+'ward_vdcmun_district_name_mapping.csv')
nepal_district    = pd.read_csv(BuildingPath+'nepal_district.csv')
nepal_district.rename(columns = {'District':'district_name'}, inplace=True)

#### Functions

* Functions to calculate distance from lat long. 
        The epicenter (latitude 28.24°N, longitude 84.75°E) of the earthquake was 77 kilometers northwest of Kathmandu at a depth of approximately 15 kilometers. 

In [5]:
from math import *
R = 6373.0

#calculate distance
def get_distance(x):
    lat_epi = radians(28.24)
    lon_epi = radians(84.75)
    lat_d   = radians(x['lat'])
    lon_d   = radians(x['long'])

    dlon = lon_d - lon_epi
    dlat = lat_d - lat_epi

    a = sin(dlat / 2)**2 + cos(lat_epi) * cos(lat_d) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return (distance)

# calculate average distance
def get_avg_distance(arr):
    all_dist = []
    if len(arr) == 1:
        arr = arr[0]
    for x in arr:
        lat_epi = radians(28.24)
        lon_epi = radians(84.75)
        lat_d   = radians(x[1])
        lon_d   = radians(x[0])

        dlon = lon_d - lon_epi
        dlat = lat_d - lat_epi

        a = sin(dlat / 2)**2 + cos(lat_epi) * cos(lat_d) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
    all_dist.append(distance)
    
    return (np.mean(all_dist))

#*ward:  chauri deurali *topo:  chaurideurali **ratio : 96
#*ward:  sunkoshi *topo:  sukoshi **ratio : 93
#*ward:  ganga jamuna *topo:  gangajamuna **ratio : 96
#*ward:  shahid lakhan *topo:  sahid lakhan **ratio : 96
#*ward:  siranchowk *topo:  siranchok **ratio : 95
# *ward:  myagang *topo:  meghang **ratio : 71
            
def get_fuzzy_string(vdcnum):
    fuzzy_low_ward = {'chumnuwri':'chumanubri','balefi':'balephi','meghang':'myagang','bhimsen': 'bhimsen thapa','indrawati':'indrawoti',
                     'tamakoshi':'likhu tamakoshi', 'nilkantha':'neelakantha','melamchi': 'melanchi', 'indrasarawor': 'indrasarowar', 
                    'baiteshwor': 'baitedhar', 'bhairabi': 'bahrabise','hetauda':'hetauda sub-metropolitian city', 'sulikot':'barpak sulikot',
                    'netrawati':'netrawati dabajon'}
    ward_main = set(ward_mapping_data['clean_vdcmun_name'].unique())
    nm2       = vdcnum.lower()
    ward_main = [nm1.lower() for nm1 in ward_main]
    if nm2 in fuzzy_low_ward.keys():
        return (fuzzy_low_ward[nm2])
    else:
        for nm1 in ward_main:
            ratio = fuzz.ratio(nm1, nm2)
            if ratio > 90:
                return (nm1)
            
# clean vdcmun name
def clean_name(name):
    change_name = {'netrawati dabajong':'netrawati dabajon', 'tamakoshi': 'likhu tamakoshi'}
    if name in change_name.keys():
        return change_name[name]
    else:
        return name

#### ward mapping and adding lat_long + distance to main file

In [6]:
district                               = ward_mapping_data.groupby(['district_id', 'district_name']).count().reset_index()[['district_id','district_name']]
dist_dict                              = dict(zip(district.district_id, district.district_name))
data['district_name']                  = data['district_id'].map(dist_dict)
nepal_district['ll_dist_district']     = nepal_district[['lat', 'long']].apply(lambda x : get_distance(x), axis=1)
data_lat_long                          = pd.merge(data, nepal_district, on='district_name', how='inner')

ward_mapping_data['clean_vdcmun_name'] = ward_mapping_data['vdcmun_name'].apply(lambda x : x.replace(' Rural Municipality', '').replace(' Municipality', '').lower())
ward_mapping_data['clean_vdcmun_name'] = ward_mapping_data['clean_vdcmun_name'].apply(lambda x : clean_name(x))
ward_mapping_data['district_name']     = ward_mapping_data['district_name'].str.lower()

In [7]:
path          = 'input_data/geoJson/'
district_geo  = pygeoj.load(path+"municipality.geojson") # to get lat-long
coordinates   = []

for feature in district_geo:
    coordinates.append(feature.geometry.coordinates)
    
with open(path+"municipality.topojson") as f:
    d = json.load(f)
    
geometries    = d['objects']['collection']['geometries']
geo_topo_dict = defaultdict(dict)

for idx, val in enumerate (geometries):
    prop                               = val['properties']
    geo_topo_dict[idx]['Name']         = prop['NAME']
    geo_topo_dict[idx]['N_ID']         = prop['N_ID']
    geo_topo_dict[idx]['Arc']          = val['arcs'] 
    geo_topo_dict[idx]['F_ID']         = prop['F_ID']
    geo_topo_dict[idx]['coordinates']  = coordinates[idx][0]
    geo_topo_dict[idx]['district_name']     = prop['DISTRICT'].lower()
    
geo_topo_df   = pd.DataFrame.from_dict(geo_topo_dict).T

In [8]:
geo_topo_df['distance_to_epicentre'] = geo_topo_df['coordinates'].apply(lambda x : get_avg_distance(x))
geo_topo_df['clean_vdcmun_name']     = geo_topo_df['Name'].apply(lambda x :get_fuzzy_string(x) if pd.notnull(x) else x)
geo_topo_df['district_name']         = geo_topo_df['district_name'].apply(lambda x : 'sindhupalchok' if x == 'sindhupalchowk' else x)

#### Create manual coordinates for vdcnum 

In [9]:
data_dict = {
'clean_vdcmun_name': ['aamachhodingmo','bahrabise','chishankhu gadhi','dhunibenshi','langtang national park','langtang national park','tarakeshwor', 'likhu tamakoshi'],
'coordinates': [[[[85.314914, 28.171318]],],[[[85.915121, 27.800278]],],[[[86.63, 27.32]],], [[[84.916667, 27.866667]],], [[[85.5, 28.25]],],[[[85.5, 28.25]],],
           [[[85.303056, 27.786667]],], [[[86.083333, 27.333333]],]],
'district_name': ['rasuwa','sindhupalchok','okhaldhunga', 'dhading','nuwakot','sindhupalchok', 'nuwakot','ramechhap']
}
manual_add = pd.DataFrame(data_dict)
manual_add['distance_to_epicentre']    = manual_add['coordinates'].apply(lambda x : get_avg_distance(x))

#### Merging ward map, topology and building_structure files

In [10]:
geo_topo_df_notnull = geo_topo_df[pd.notnull(geo_topo_df['clean_vdcmun_name'])][['coordinates', 'district_name','distance_to_epicentre', 'clean_vdcmun_name']]
geo_topo_manual     = pd.concat([geo_topo_df_notnull, manual_add])
ward_topo           = pd.merge(ward_mapping_data, geo_topo_manual, on=['district_name','clean_vdcmun_name'], how='left')
data_with_distance  = pd.merge(ward_topo[['distance_to_epicentre', 'ward_id']], data, on=['ward_id'], how='inner')

#### convert damage grade to numeric

In [11]:
damage_dict          ={'Grade 3':2, 'Grade 5':3, 'Grade 2':1, 'Grade 1':0, 'Grade 4':3}
data_with_distance['damage_grade_num'] = data_with_distance['damage_grade'].map(damage_dict)

In [9]:
damage_dict          ={'Grade 3':'Grade 3', 'Grade 5':'Grade 4', 'Grade 2':'Grade 2', 'Grade 1':'Grade 1', 'Grade 4':'Grade 4'}
data_with_distance['damage_grade_vis'] = data_with_distance['damage_grade'].map(damage_dict)

In [28]:
# damage_dict          ={'Grade 3':'Medium', 'Grade 5':'High', 'Grade 2':'Low', 'Grade 1':'Low', 'Grade 4':'High'}
# data_with_distance['damage_grade_lmh'] = data_with_distance['damage_grade'].map(damage_dict)

#### Saving final output

In [12]:
data_with_distance.to_csv('clean_data/building_structure_ownership_withdistance_283.csv', index=False)

#### Splitting data into 70 % train and 30 %test

In [39]:
train, test = train_test_split(data_with_distance, test_size=0.3, random_state=0, stratify=data_with_distance['damage_grade_num'])

In [43]:
train.to_csv('clean_data/train_withdistance.csv', index=False)
test.to_csv('clean_data/test_withdistance.csv', index=False)

In [44]:
train['damage_grade_num'].value_counts()

3    320456
2     95065
1     60835
0     54909
Name: damage_grade_num, dtype: int64

In [45]:
test['damage_grade_num'].value_counts()

3    137339
2     40742
1     26072
0     23532
Name: damage_grade_num, dtype: int64